# Assignment 3
## COGS 189: Brain Computer Interfaces
## Winter 23, Instructor: Alessandro D'Amico
### Due Date: 05Feb2023
***
## Overview:
For this assignment (tutorial), we will be working with a BCI competition dataset, with the primary goal of classifying target vs non-target trials from a P300 speller. The data we will be using comes from the 2nd Berlin BCI Competition, and was provided by Wadsworth Center, NYS Department of Health (Jonathan R. Wolpaw). Please go to the following link: 
http://www.bbci.de/competition/ii/
<br><br>
We will be working with **Dataset IIb**. Please click on the description of this dataset (pdf format is superior) and read through the page in order to answer the following questions.

Here is a direct link: http://www.bbci.de/competition/ii/albany_desc/albany_desc_ii.pdf
***
***Question 1:*** What is the *dt* (in ms; round to the nearest ms) for the row/column intensification? 
<br>*(Hint: dt is the time between samples. For example, the dt at 1000 Hz = 1ms, 500 Hz = 2ms, 200 Hz = 5ms)*<br>
***Question 2:*** How long (in ms) was each row/column intensified?<br>
***Question 3:*** How much time (in ms) was there between each row/column intensification?<br>
***Question 4:*** What is the sampling rate of the EEG data? **(DO NOT INCLUDE Hz IN YOUR ANSWER)**<br>
***Question 5:*** Within a single block of 12 trials, how many times is each character illuminated?
<br>*(Hint: A single block of 12 trials consists of each of the 6 rows being illuminated and each of the 6 columns being illuminated)*
***
<img src="assets/P300Matrix.gif">

*I created the animation following the description of the dataset, although this was not what participants actually saw. As you can tell, these stimuli are being presented in rapid succession. The participant was instructed to only focus on one character at a time. This shows 15 blocks of 12 intensifications which was the number of trials per letter described in the description*
***
***Note:*** The description of this dataset does not detail what the reference electrode was; this is bad practice. It took quite a long time to find, but I believe the reference used was the right earlobe as described in this paper:<br> https://hal.archives-ouvertes.fr/file/index/docid/521054/filename/krusienski_jne_2006.pdf
<br>
This is the same group and the parameters for data collection seem identical which is why I believe the descriptions of that paper apply to the data we'll be working with. They also mention they have already filtered their data with a 0.1 - 60 Hz bandpass filter. We will still apply a bandpass filter with a 0.1 Hz low cut off.
***
The owners of this data provided us with useful descriptions of the data, along with some matlab code. We will use their documentation to epoch the data so that we can train our classifier. We will be using linear discriminant analysis (LDA) for this problem, as it performs very well on P300 paradigms. Furthermore, this algorithm is rather intuitive. We will be using ```sklearn``` as it will make this assignment easier to follow.

## Section 1: Setup
Before we can begin processing our data, we must first import some useful packages. <br>
- `numpy` is used widely for processing numerical data, and supports matrix operations natively. We will be using NumPy arrays to store our data.
- `scipy` contains useful functions to calculate filter coefficients and carry out filtering. We will be using SciPy filters to clean our EEG data.
- `matplotlib` and `seaborn` are used to create plots. We will use them to visually explore and examine our data.
- `sklearn` is used for machine learning. We will use this to train our classifier and make predictions.

In [ ]:
import numpy as np                                      
import matplotlib.pyplot as plt                         
import matplotlib.patches as patches
import seaborn as sns
import scipy.signal as signal 
from scipy.io import loadmat
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from pathlib import Path

## Section 2: Pre-processing

### Step 1: Design a filter
We first create some variables to store our sampling rate (`fs`), the amount of time between samples (`dt`), and various variables for our filter parameters. We'll be using an FIR windowed bandpass filter with 0.1 Hz and 30 Hz cutoffs. Since we'll be filtering on epoched data, we have to utilize a relatively low order/number of taps. We'll utilize the function `filtfilt()` in order to filter the data forwards and backwards (non-casual). This is a zero-phase filter, meaning our ERPs will remain precisely synchronized and our component of interest shouldn't move as a result of filtering.
<br><br>
Here is the summary of our filter design:
- `Filter..................: Hamming Windowed-Sinc (FIR); zero-phase, non-causal`
- `Number of coefficients..: 31`
- `High pass...............: 0.1 Hz (low cut of our bandpass)`
- `Low pass................: 30 Hz (high cut of our bandpass)`

In [ ]:
# Define our filter variables
fs = 240.0                      # Hz; sampling rate
dt = 1000. / fs                 # ms; time between samples
sdt = np.round(dt).astype(int); # rounded dt so that we can index samples
hp = 0.1                        # Hz; our low cut for our bandpass
lp = 30.                        # Hz; our high cut for our bandpass
num_taps = 31                   # Number of taps/coefficients of FIR filter

# Create our filter coefficients
# Note: by defining 'fs' we don't divide our windows by the Nyquist
# Note: for FIR filters, a is always 1
b = signal.firwin(numtaps=num_taps, cutoff=[hp, lp], pass_zero='bandpass', fs=fs)
a = 1

### Step 2: Determine our epoch, baseline, and window of focus
For the epoch, we'll plot the activity from 0ms (stimulus onset) to 700 ms (after onset) to ensure we have a waveform similar to the plot provided by the authors. For the baseline, we'll use 0ms to 100 ms after stimulus onset. This baseline window was chosen because every epoch contains information from the previous stimulus due to the small interstimulus interval (ISI). For the window we will select our ERP component from, we'll use 200 ms after the stimulus to 400 ms after the stimulus. This choice is consistent with the theoretical expected onset of the P3b, and minimizes signals from previous and subsequent stimuli. For readibility:
- `Epoch...: 0 - 700 ms`
- `Baseline: 0 - 100 ms`
- `ERP.....: 200 - 400 ms`

In [ ]:
# Define ERP-related variables
epoch_start = 0    # ms
epoch_end = 700    # ms
baseline_start = 0 # ms
baseline_end = 100 # ms
erp_start = 200    # ms
erp_end = 400      # ms

# Let's translate these from time into index space to save time later
e_s = np.round(epoch_start / sdt).astype(int)     # epoch start
e_e = np.round(epoch_end / sdt).astype(int)       # epoch end
bl_s = np.round(baseline_start / sdt).astype(int) # baseline start
bl_e = np.round(baseline_end / sdt).astype(int)   # baseline end
erp_s = np.round(erp_start / sdt).astype(int)     # ERP component window start
erp_e = np.round(erp_end / sdt).astype(int)       # ERP component window end

# We'll also create our channel labels here since the document only provided a figure
channels = np.array(['FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6',
                     'C5',  'C3',  'C1',  'Cz',  'C2',  'C4',  'C6',
                     'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6',
                     'Fp1', 'Fpz', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8',
                     'F7',  'F5',  'F3',  'F1',  'Fz',  'F2',  'F4',  'F6', 'F8',
                     'FT7', 'FT8', 'T7',  'T8',  'T9',  'T10', 'TP7', 'TP8', 
                     'P7',  'P5',  'P3',  'P1',  'Pz',  'P2',  'P4',  'P6', 'P8',
                     'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'O1',  'Oz',  'O2', 'Iz'])

### Step 3: Load and epoch our data

We are now ready to extract our signals of interest. All of our data is contained in the *data* subfolder within this directory. Inside are 19 ```.mat``` files. These files were all recorded from the same subject performing unique runs of copy spelling. Copy spelling is when a subject is told which words/letters they will be spelling so that we can label their data and train a classifier to be used during free spelling. In this case, as mentioned in the document, our training data have the prefix ***AAS010*** and ***AAS011*** , while our test (unlabeled) data have the prefix ***AAS012***
<br><br>
Using our previously defined epoch-related and filter-related variables, we will iterate through all of our files, load them in, extract the relevant pieces of information, process our signals, and store them in easily accessed structures. Our training data, ```train_data```, will also be accompanied by ```train_labels``` which will be a binary vector with 0 indicating non-target and 1 indicating a target, and ```train_markers```, which will contain the row/column which was illuminated during the trial. For our testing data, ```test_data``` will contain our epochs of interest, but since we have no labels for this data, it will only be accompanied by ```test_markers```.
<br><br>
It's important to note that we will be filtering our EEG signals as we append them to their respective structures.
<br><br>
***Since the code below is written to be maximally readable, it will be relatively (aka very) slow.***

In [ ]:
# Create empty lists to store our data. We'll conver them into np.arrays at the end
train_data = []
train_labels = []
train_markers = []
test_data = []
test_markers = []

# Load all data (concatenate train data, keep test separate)
pfx = 'AAS0'
dir_ = './data/'
train = [10, 11]
S = [10, 11, 12]
R = [x for x in range(1, 9)]
for s in S:
    for r in R:
        #fp = Path(dir_ + pfx + '%dR0%d.mat' % (s, r))
        fp = Path(f'{dir_}{pfx}{s}R0{r}.mat')
        if fp.exists() and fp.is_file():
            # If file exists, load, process, append
            print(f'Processing: {fp}')
            x = loadmat(fp)

            # All we need are the EEG data (signal) and markers (StimulusCode) out of this
            data_   = np.array(x['signal'])        # dim (num(time_points) x num(channels))
            mrks_   = x['StimulusCode'].flatten(); # dim (num(time_points) x 1)

            # Create a stimulus onset array
            onsets = np.zeros_like(mrks_)
            for i in range(mrks_.shape[0]):
                # Roughly equivalent to positive first derivative, but easier to read
                if mrks_[i] != 0 and mrks_[i-1] == 0:
                    onsets[i] = 1
                    
            # Determine all indices where onsets occur
            onsets_ix = np.where(onsets == 1)[0]
        
            # Iterate through onsets, epoch, filter, append
            for i in range(len(onsets_ix)):
                t0 = onsets_ix[i]
                
                # Epoch and correct DC offset of signal
                data = data_[e_s+t0:t0+e_e, :] - np.mean(data_[e_s+t0:t0+e_e, :], 0)
                
                # Filter the epoch (non-causal, zero-phase filter)
                data = signal.filtfilt(b, a, data, axis=0)
                
                # Now let's baseline correct
                data = data - np.mean(data[bl_s+np.abs(e_s):np.abs(e_s)+bl_e, :], 0)
                
                # Append data to correct locations
                if s in train:
                    train_data.append(data)
                    train_markers.append(mrks_[t0])
                    train_labels.append(x['StimulusType'].flatten()[t0]) # target or non-target
                else:
                    test_data.append(data)
                    test_markers.append(mrks_[t0])
                    
print('Processing completed!')

# Convert all of our lists into numpy arrays
train_data = np.array(train_data)
train_labels = np.array(train_labels)
train_markers = np.array(train_markers)
test_data = np.array(test_data)
test_markers = np.array(test_markers)

Now that we have our epoched signals, let's divide our ```train_data``` into target and non-target categories for plotting.

In [ ]:
# Seperate target and non-target for plotting
tar     = train_data[np.where(train_labels == 1)[0], :, :]
non_tar = train_data[np.where(train_labels == 0)[0], :, :]

print(f'We have {tar.shape[0]} target trials')
print(f'We have {non_tar.shape[0]} non-target trials')

# We'll take the average of all trials to create an averaged ERP
tar_avg     = np.mean(tar, 0)
non_tar_avg = np.mean(non_tar, 0)

# Define channel of interest and create an array of time points
chan = 'Pz' # let's plot Pz
ch = np.where(channels == chan)[0][0]
times = np.linspace(epoch_start, epoch_end, train_data.shape[1])

# Initialize plot and calculate min and max y value
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 6))
min_y = min(np.min(tar_avg), np.min(non_tar_avg))
max_y = max(np.max(tar_avg), np.max(non_tar_avg))

# Plot x and y axes
plt.plot([np.min(times), np.max(times)], [0, 0], color='k');  # x-axis
plt.plot([0, 0], [min_y, max_y], color='k');                  # y-axis

# Plot our averaged ERPs
plt.plot(times, tar_avg[:, ch], 'b', linewidth=4)
plt.plot(times, non_tar_avg[:, ch], 'r', linewidth=4)

# Highlight the baseline window and window of interest of our ERP
baseline = patches.Rectangle([baseline_start, min_y], baseline_end, np.abs(min_y)+max_y, 
                             color='c', alpha=0.2)
erp_win = patches.Rectangle([erp_start, min_y], erp_end-erp_start, np.abs(min_y)+max_y, 
                             color='lime', alpha=0.2)

# Add our baseline and window of interest highlights
ax.add_patch(baseline)
ax.add_patch(erp_win)

# Manually create legends since patches will corrupt default handles
legend_ = [patches.Patch(color='b', label = 'Target (oddball)'),
           patches.Patch(color='r', label = 'Non-target (standard)')]

# Finalize plot and set a high DPI for a crisp, hi-res figure
plt.xlabel('Time (msec)');
plt.ylabel('Amplitude (A/D Units)');
plt.legend(handles=legend_, loc='upper right');
plt.title(f'Event Related Potentials at Channel {chan}');
fig.set_dpi(216);
plt.show();

**Step 3: Determine how many signals to extract**<br>
There is a lot of mutual information in our data, and it would not be productive to export all of it, specifically because it would take more time to train/cross-validate our model. If you're curious, try modifying ```channel``` to examine the ERPs at different locations. You'll find that there are a lot of simillarities between the waveforms of neighboring channels. Neighboring time points are also very similar and we could faithfully recreate our waveform using significantly fewer samples. With this in mind, we can reduce our dimensionality quickly by sub-sampling our data in time, while preseving all 64 channels. One efficient way to sub-sample in a scenario like this is to calculated the ***windowed means*** of our signal of interest.
<br><br>
<img src="assets/img1.png">
<center>A simple illustration of sub-sampling (more generalized than windowed means)</center>
<br><br>
To calculate our windowed means, we can divide our data into equally sized chunks and calculate the means. In some instances it can be beneficial to have overlap between windows, but for this case we can keep the windows totally seperated. Let's extract 5 points per waveform per channel.

In [ ]:
# Let's compute the windowed means within erp_start and erp_end
num_points = 5; # we will divide our window into num_points means

# Define a simple windowed means function
def wm(x, start, end, num_points):
    num_trials = x.shape[0] # assumes first dem is numb observations
    w = np.round((start+end)/num_points).astype(int)
    y = np.zeros((num_points, x.shape[-1], num_trials)) # assumes num chans as last dimension
    for i in range(0, num_points):
        s = start + (w * i)
        e = end   + (w * i)
        y[i, :, :] = np.mean(x[:, s:e, :], 1).T
    return y

# Combine into a single train variable. Also create labels
X_train    = wm(train_data, erp_s, erp_e, num_points)
markers_train = np.vstack((train_labels, train_markers)).T
y = train_labels

# Now let's compute windowed means of our test data
X_test = wm(test_data, erp_s, erp_e, num_points)
markers_test = test_markers

# Let's print out the shapes of our data
print(f'X_train shape is: {X_train.shape}')
print(f'y shape is......: {y.shape}')
print(f'X_test shape is.: {X_test.shape}')

As we can see, the dimensions of ```X_train``` are 5 x 64 x 7561. We have 5 samples per observation (since we 10-times subsample a window of ~50 samples), 64 channels, and 7561 unique observations. Similarly, we have 5580 unique observations for our test data, ```X_test```. We must restructure our data so that they can be passed into our classifier. Luckily, our 5 x 64 features can be flattened, and we can subsequently transpose our matrix so that the first dimension matches our label vector (this is the format ```sklearn``` expects). We will do the same flattening and transposing on our test data.

In [ ]:
# Since our X is 3D, we must flatten our data. We will then transpose it for sklearn
X_train = X_train.reshape(-1, X_train.shape[-1]).T
X_test = X_test.reshape(-1, X_test.shape[-1]).T

# Let's print out the new shape
print('X_train shape is now: ' + str(X_train.shape))
print('X_test  shape is now: ' + str(X_test.shape))

Now that our data are correctly formatted, we can train a model! We will first choose our classifier. One standard choice for classifying P300 data is via linear discriminant analysis (LDA) which has been discussed in class. sklearn supports LDA (and its quadratic counterpart QDA) which makes the process rather simple. 
<br><br>
We will be using the least-squares solver, which tends to be more efficient for higher dimensional feature vectors.
<br><br>
We will shrink the covariance matrix, which is important when we have high-dimensional data with relatively few observations, and we will determine this parameter automatically. This process is called shrinkage.<br>
See: https://scikit-learn.org/stable/auto_examples/classification/plot_lda.html#sphx-glr-auto-examples-classification-plot-lda-py

In [ ]:
# Train our classifier (this may take a while via JupyterHub)
clf = LinearDiscriminantAnalysis(solver='lsqr',  shrinkage='auto').fit(X_train, y)

Let's do 5-fold cross validation to determine the accuracy of our model<br>
***Note: This may take a few minutes to run on slower machines***

In [ ]:
# Let's do 5-fold cross validation
nfolds = 5;
score = cross_val_score(clf.fit(X_train, y), X_train, y, cv=nfolds)

# We will print out the mean score
print(f'Mean classifier accuracy: {np.mean(score)}')

Since we know that the non-target stimuli are considerably more common *(10/12, 83%)*, we would expect to get around 80% accuracy by just saying everything is a non-target. In order to examine our model more carefully, we will use the receiver operating characteristic curve (ROC) and calculate the area under the curve (AUC) to see how often our classifier is correctly labeling the data. We will also use 5-fold cross validation for this method.<br>
https://en.wikipedia.org/wiki/Receiver_operating_characteristic <br>
https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/ <br>
https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html#

In [ ]:
# Modified from:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html
cv = StratifiedKFold(n_splits=nfolds)
X = X_train
tprs = [];                         # true positive rates
aucs = [];                         # areas under curve
mean_fpr = np.linspace(0, 1, 100); # false positive rates

i = 0
for train, test in cv.split(X, y):
    probas = clf.fit(X[train], y[train]).predict_proba(X[test])
    fpr, tpr, th = roc_curve(y[test], probas[:, 1])
    tprs.append(np.interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    i += 1

Now that we have created our ROC and calculated the AUC, let's plot the curve and include the AUCs for our model

In [ ]:
# Modified from:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
         label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs);

plt.plot(mean_fpr, mean_tpr, color='b',
         label=f'Mean ROC (AUC = {mean_auc:0.2f} $\pm$ {std_auc:0.2f})',
         lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='b', alpha=.2,
                 label='$\pm$ 1 std. dev.')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'ROC of Least-Squares LDA ({nfolds}-fold CV)')
plt.legend(loc='lower right')
plt.show()

It seems that the AUCs agree with our 5-fold cross validation above! We should get fairly good accuracy on our testing data if we haven't been overfitting the data!

Let's now use our classifier (trained on all of our training data) and use it to predict outputs from the test data.

In [ ]:
# Make predictions on our dataset 
conf = clf.decision_function(X_test) # predicted confidence score (log-posterior)
pred = clf.predict(X_test)           # predicted label (we won't actually use this)

At this stage, we have trained a model, and used it to make predictions on individual observations (target vs non-target). We will be using the confidence score of each classification to now determine which character the subject was attending to. The larger the score, the more confident the classification was that the observation was a flashed target.
<br><br>
Let's create a numpy array to mimic the character matrix described in the data description, in order to efficiently determine which character our classifier predicted
<img src="assets/img2.png">

In [ ]:
# Here are the translations for the labels:
# Columns 0-5 -> Labels 1-6
# Rows 0-5 -> Labels 7-12
# Let's create this matrix
#                         1    2    3    4    5    6
char_matrix = np.array([['A', 'B', 'C', 'D', 'E', 'F'], # 7
                        ['G', 'H', 'I', 'J', 'K', 'L'], # 8
                        ['M', 'N', 'O', 'P', 'Q', 'R'], # 9
                        ['S', 'T', 'U', 'V', 'W', 'X'], # 10
                        ['Y', 'Z', '1', '2', '3', '4'], # 11
                        ['5', '6', '7', '8', '9', '0']])# 12 

We know that for each character, each row and column was flashed once, for a total of 12 flashes per block and this was repeated 15 times. This means that the first 12x15 data points correspond to the first character, the 2nd 12x15 data point correspond to the second character, etc.
<br><br>
Let's examine the first character from the test data

In [ ]:
# Define some useful variables
char_breaks = 15 * 12 # number of stimuli per character
c = 1;                # which character we want

# Grab our first character's marker, prediction, and confidence
char1_mrks = markers_test[char_breaks*(c-1):char_breaks*c]
char1_pred = pred[char_breaks*(c-1):char_breaks*c]
char1_conf = conf[char_breaks*(c-1):char_breaks*c]

# Let's merge these
char1 = np.array([char1_mrks, char1_pred, char1_conf]).T

We now have an array `char1` which contains 180 stimuli, their marker (which row/col), their prediction (0, 1 unused), and the confidence of the prediction (float). Let's sort our `char1` by confidence in order to determine which row and column gives us the character that we believe to be the subject's target.

In [ ]:
# Alright, now let's sort them by the 3rd column, the confidence
# Nice 1-liner provided by https://stackoverflow.com/questions/2828059/sorting-arrays-in-numpy-by-column
char1_sort = char1[char1[:,2].argsort()[::-1]]

# Let's print out the first 5 results
print(char1_sort[0:5, :])

Of all the flashes, the one with the highest confidence (that it contains a target) is the marker `'6'` which corresponds to the last column.<br>
Our second highest confidence is the marker `'7'` which corresponds to the first row.<br> 

We could simply take the column/row that has the highest single-flash confidence and consider the target character prediction to be the character in that highest confidence row and highest confident column. However, it's possible that those values are outliers. In order to make better use of all the data, we will take the mean of every marker's (row's and column's) confidence, and use the row and column with the highest average confidence to determine our character.

In [ ]:
# Now we will store the average confidence value for every row/column
x = np.zeros(shape = (12, 1))
for i in range(0, x.shape[0]):
    x[i] = np.mean(char1_sort[char1_sort[:, 0] == i + 1, 2])
    
# Now let's print our character
row = np.argmax(x[6:])
col = np.argmax(x[0:6])
our_char = char_matrix[row, col]
print(f'Our character is: {our_char}')

Just for fun, let's visualize our confidence intervals over each letter by utilizing a heatmap. 

In [ ]:
# Define useful plotting variables
rows = 6
columns = 6
fontsize = 20

# Initialzie figure/axes
fig, ax = plt.subplots(ncols=1, nrows=1, constrained_layout=False, figsize=(8, 6), 
                       facecolor='white', edgecolor='white', dpi=100)

# Let's turn x into a matrix where we take each character's avg CI from row/col
x_mat = np.zeros((rows, columns))
for j in range(0, columns):
    for i in range(columns, rows+columns):
        x_mat[i-columns, j] = np.mean((x[i][0], x[j][0]))

# Plot confidence interval heatmap
sns.heatmap(x_mat, cmap=sns.color_palette('vlag', as_cmap=True), ax=ax, 
            cbar_kws={'label':'Average Row/Column Confidence Score'})

# Plot each character (adapted from https://stackoverflow.com/questions/73071524/)
for row in range(rows):
    for column in range(columns):
        # Plot each individual character over the heatmap
        plt.text((column+0.5), (row+0.5),
                char_matrix[row][column], fontsize=fontsize, ha='center', va='center')

# Hide axes and borders
ax.spines['bottom'].set_color('white')
ax.spines['top'].set_color('white') 
ax.spines['right'].set_color('white')
ax.spines['left'].set_color('white')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

plt.show()

Excellent! Our classifier predicts that the first character the subject attended to was the letter `'F'` and the heatmap shows that the letter `'F'` has the highest average confidence interval. Let's now use this logic and apply it iteratively to each character in the test data, and print out every character the user attended to:

In [ ]:
# Main loop: Iterate through test data by chunking every 15x12 trials per character
#  Nest 1: Calculate mean of character
char_breaks = 15*12
output_string = f''
for c in range(1, int(X_test.shape[0] / char_breaks) + 1):
    # Lets break up test into char 'c'
    temp_mrks = markers_test[char_breaks * (c - 1):char_breaks * c]
    temp_pred = pred[char_breaks * (c - 1):char_breaks * c]
    temp_conf = conf[char_breaks * (c - 1):char_breaks * c]

    # Let's merge these and sort by highest confidence
    temp_char = np.array([temp_mrks, temp_pred, temp_conf]).T
    temp_char = temp_char[temp_char[:, 2].argsort()[::-1]]
    
    # Create our averages
    x = np.zeros(shape = (12, 1))
    for i in range(0, x.shape[0]):
        x[i, 0] = np.mean(temp_char[temp_char[:, 0] == i + 1, 2])
        
    # Now let's print our character
    row = np.argmax(x[6:,  0])
    col = np.argmax(x[0:6, 0])
    our_char = char_matrix[row, col]
    output_string += f'{our_char} '
    
# Print our result
print(output_string)

Since this is a competition dataset, we were obviously not given the labels to our test set. However, since this competition is quite old (2002), the results are public!

5 teams got first place in this competition, each with test character 100% accuracy. The added challenge was to minimize the amount of flashes each character was required, but for simplicity we will not do this.

Since all of these teams were able to determine the desired characters with 100% accuracy, we can look at one of their write-ups to determine the true labels of the test set ***(click on link below and scroll to the bottom)***.

The approach we took for this assignment was influenced by (but not a direct copy of): http://www.bbci.de/competition/ii/results/tax_iib_desc.pdf
<br>
***
***Question 6:*** What percentage of our test characters were correctly classified?
***